In [1]:
import numpy as np
import torch
import json
import matplotlib.pyplot as plt

In [2]:
codenames_file = '../data/processed_data/codenames_vecs.json'
dictionary_file = '../data/processed_data/dictionary_vecs.json'

In [39]:
class Codenames:
    def __init__(self,
                codenames_file='../data/processed_data/codenames_vecs.json',
                dictionary_file='../data/processed_data/dictionary_vecs.json',
                threshold=0.1
    ):
        self.codenames_words, self.codenames_vecs = self.load_data(codenames_file)
        self.dictionary_words, self.dictionary_vecs = self.load_data(dictionary_file)
        self.threshold = threshold
        self.initiate_game()
        self.display_board()

    def load_data(self, file):
        with open(file) as f:
            content = json.load(f)
        words = np.array(list(content.keys()))
        vecs = np.array(list(content.values()))
        return words, vecs

    def initiate_game(self):
        self.cosine_sim_mat = np.matmul(self.dictionary_vecs, self.codenames_vecs.T) # shape = (NUM_DICT, NUM_CODENAMES)
        game_words = np.random.choice(range(len(self.codenames_words)), 25)
        self.remove_game_words_from_dict(game_words)
        self.team_a = game_words[:9]
        self.team_b = game_words[9:17]
        self.neutral = game_words[17:24]
        self.black = game_words[24:25]
        self.game_word_ids = np.random.choice(game_words, 25, replace=False)
        return None
    
    def remove_game_words_from_dict(self, game_words):
        lst_to_remove = []
        for idx in game_words:
            match_num = np.where(self.dictionary_words==self.codenames_words[idx])[0]
            if len(match_num) > 0:
                lst_to_remove.append(match_num[0])
        self.cosine_sim_mat = np.delete(self.cosine_sim_mat, lst_to_remove, 0)
        self.dictionary_words = np.delete(self.dictionary_words, lst_to_remove)
        self.dictionary_vecs = np.delete(self.dictionary_vecs, lst_to_remove)
        return None
        
    def display_board(self):
        print(np.array(self.codenames_words[self.game_word_ids]).reshape(5,5))
        return None
    
    def get_team_a_words(self):
        return self.codenames_words[self.team_a]
    
    def get_team_b_words(self):
        return self.codenames_words[self.team_b]
    
    def compute_score(self):
        ########
        # TODO #
        ########
        ret = []
        enemy_words = np.concatenate((self.team_a, self.neutral, self.black))
        for i in range(len(self.dictionary_words)):
            lowest_bound_ai = np.max(
                self.cosine_sim_mat[i, enemy_words] + [self.threshold]
            )
            suggested_count = np.sum(self.cosine_sim_mat[i, self.team_b] > lowest_bound_ai)
            weighted_score = np.sqrt(lowest_bound_ai) * suggested_count
            ret.append([lowest_bound_ai, suggested_count, weighted_score])
        ret = np.array(ret)
        return ret
    
    def find_optimal_num_guess(self, idx):
        ########
        # TODO #
        ########
        print(self.cosine_sim_mat[idx, self.team_a])
        print(self.cosine_sim_mat[idx, self.team_b])
        print(self.cosine_sim_mat[idx, self.neutral])
        print(self.cosine_sim_mat[idx, self.black])
        
    def ai_guess(self):
        scores = self.compute_score()
        idx = scores[:, 2].argmax()
        print(scores[idx])
        self.find_optimal_num_guess(idx)
        return self.dictionary_words[idx]

In [40]:
a = Codenames()

[['mexico' 'england' 'racket' 'swing' 'tick']
 ['lock' 'shark' 'swing' 'pie' 'sink']
 ['stream' 'suit' 'line' 'horseshoe' 'march']
 ['screen' 'shark' 'brush' 'pilot' 'saturn']
 ['alien' 'mole' 'cotton' 'center' 'brush']]


In [41]:
a.ai_guess()

[0.21984532 3.         1.40662996]
[ 0.11951952  0.06091695  0.1160201   0.1161056   0.05166761  0.11984532
  0.07540869 -0.03486341  0.03746789]
[0.19827251 0.11951952 0.11984532 0.17430963 0.09723017 0.34918793
 0.25018416 0.27870613]
[0.1024112  0.10935318 0.10991018 0.11148128 0.03746789 0.05803488
 0.10207111]
[0.05872795]


'courts'

In [42]:
a.get_team_b_words()

array(['england', 'swing', 'brush', 'center', 'pie', 'racket',
       'horseshoe', 'suit'], dtype='<U11')